# GBM
- Data Split 
    - Apply 80/20 split by Ticker
    - For calculating performance, use 5 fold cross validation (RMSE, Accuracy (1 if positive, 0 if not))
- Build GBM by Ticker

In [2]:
import xgboost
import pandas as pd
import numpy as np

In [3]:
df = pd.read_parquet("../data/processed/daily_with_finbert_and_indicators.parquet")

In [5]:
df.shape

(2490, 805)

In [7]:
df.head()

,ticker,date,open,high,low,close,volume,return_t+1,headlines,finbert_0,...,open_smoothed_alpha:0.9,close_smoothed_alpha:0.25,close_smoothed_alpha:0.75,close_smoothed_alpha:0.9,return_t+1_smoothed_alpha:0.25,return_smoothed_alpha:0.25,return_t+1_smoothed_alpha:0.75,return_smoothed_alpha:0.75,return_t+1_smoothed_alpha:0.9,return_smoothed_alpha:0.9
0,AAPL,2024-05-01,169.580002,172.710007,169.110001,169.300003,50383100.0,0.022032,[],0.000000,...,169.580002,169.300003,169.300003,169.300003,0.022032,NaN,0.022032,NaN,0.022032,NaN
1,AAPL,2024-05-02,172.509995,173.419998,170.889999,173.029999,94214900.0,0.059816,[],0.000000,...,172.216995,170.232502,172.097500,172.656999,0.031478,0.022032,0.050370,0.022032,0.056038,0.022032
2,AAPL,2024-05-03,186.649994,187.000000,182.660004,183.380005,163224100.0,-0.009107,[],0.000000,...,185.206694,173.519378,180.559379,182.307704,0.021332,0.031478,0.005762,0.050370,-0.002592,0.056038
3,AAPL,2024-05-06,182.350006,184.199997,180.419998,181.710007,78569700.0,0.003797,[],0.000000,...,182.635675,175.567035,181.422350,181.769776,0.016948,0.021332,0.004289,0.005762,0.003158,-0.002592
4,AAPL,2024-05-07,183.449997,184.899994,181.320007,182.399994,77305800.0,0.001864,[From Apple to Wayfair: A Lot of Earnings Resu...,0.030524,...,183.368565,177.275275,182.155583,182.336972,0.013177,0.016948,0.002470,0.004289,0.001994,0.003158


In [6]:
df.tail()

,ticker,date,open,high,low,close,volume,return_t+1,headlines,finbert_0,...,open_smoothed_alpha:0.9,close_smoothed_alpha:0.25,close_smoothed_alpha:0.75,close_smoothed_alpha:0.9,return_t+1_smoothed_alpha:0.25,return_smoothed_alpha:0.25,return_t+1_smoothed_alpha:0.75,return_smoothed_alpha:0.75,return_t+1_smoothed_alpha:0.9,return_smoothed_alpha:0.9
2485,XOM,2025-04-23,108.809998,109.300003,106.470001,107.370003,14235700.0,0.011735,[Will Exxon (XOM) Beat Estimates Again in Its ...,0.067738,...,108.536702,106.228441,107.430955,107.434789,0.005872,0.003918,0.008416,-0.001541,0.010030,-0.005316
2486,XOM,2025-04-24,107.970001,108.879997,106.959999,108.629997,9849600.0,-0.000552,[Exxon Mobil (NYSE:XOM) Partners With Calpine ...,0.076386,...,108.026671,106.828830,108.330237,108.510476,0.004266,0.005872,0.001690,0.008416,0.000506,0.010030
2487,XOM,2025-04-25,108.139999,108.760002,107.419998,108.570000,11245800.0,0.000553,[Guyana Q1 oil production sees 3% YoY increase...,-0.027245,...,108.128667,107.264123,108.510059,108.564047,0.003338,0.004266,0.000837,0.001690,0.000548,0.000506
2488,XOM,2025-04-28,108.750000,109.250000,107.680000,108.629997,13890900.0,-0.002485,[Oil Slumps as Traders Await Next Moves in Chi...,-0.336978,...,108.687867,107.605591,108.600013,108.623402,0.001882,0.003338,-0.001655,0.000837,-0.002182,0.000548
2489,XOM,2025-04-29,107.739998,108.830002,107.519997,108.360001,9483100.0,-0.025194,"[Exxon Mobil Q1 Earnings Boosted by Upstream, ...",-0.062517,...,107.834785,107.794194,108.420004,108.386341,-0.004887,0.001882,-0.019309,-0.001655,-0.022893,-0.002182


### Apply data split

### Build initial model on Apple 
- Use ARIMA as a baseline (and potentially use as a feature)